In [ ]:
def predict_algorithm(stock_name):

    # Importing the libraries
    import numpy as np
    import matplotlib.pyplot as plt
    import pandas as pd
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.layers import LSTM
    from keras.layers import Dropout
    from sklearn.metrics import mean_squared_error
    from math import sqrt

    print("Libraried Imported Successfully!")
    # Importing the training set
    dataset_train = pd.read_csv('C:/Users/akshaf/Desktop/Final Project/NSEStock_train.csv')
    dataset_train
    dataset_train.set_index("SERIES", inplace = True)
    training_set_1 = dataset_train.loc[dataset_train["SYMBOL"] == stock_name, 'OPEN'].values
    training_set = training_set_1.reshape(-1,1)
    
    print("Training set created")
    
    # Feature Scaling
    from sklearn.preprocessing import MinMaxScaler
    sc = MinMaxScaler(feature_range = (0, 1))
    training_set_scaled = sc.fit_transform(training_set)
    print("Training set Scaled")
    
    # Creating a data structure with 40 timesteps and 1 output
    
    timestamp = 40
    
    X_train = []
    y_train = []
    for i in range(timestamp, len(training_set)):
        X_train.append(training_set_scaled[i-timestamp:i, 0])
        y_train.append(training_set_scaled[i, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)
    
    print("X_train and y_train created")
    
    # Reshaping
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    
    # Part 2 - Building the RNN
    
    # Initialising the RNN
    regressor = Sequential()
    print("Regressor Created")
    
    # Adding the first LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    regressor.add(Dropout(0.2))
    print("First LSTM layer created")
    
    # Adding a second LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))
    print("Second LSTM layer created")
    
    # Adding a third LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(0.2))
    print("Third LSTM layer created")
    
    # Adding a fourth LSTM layer and some Dropout regularisation
    regressor.add(LSTM(units = 50, return_sequences = False))
    regressor.add(Dropout(0.2))
    print("Fourth LSTM layer created")
    
    # Adding the output layer
    regressor.add(Dense(units = 1))
    print("Output layer added")
    
    # Compiling the RNN
    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
    
    # Fitting the RNN to the Training set
    regressor.fit(X_train, y_train, epochs = 100, batch_size = 10)
    #print("RNN Regressor fitted on Training set")
    
    # Part 3 - Making the predictions and visualising the results
    
    # Getting the real stock price of in Nov-Dec 2017
    dataset_test = pd.read_csv('C:/Users/akshaf/Desktop/Final Project/NSEStock_Test.csv')    
    #dataset_test.set_index("SERIES", inplace = True)

    real_stock_price = dataset_test.loc[dataset_test["SYMBOL"] == stock_name, 'OPEN'].values
    print("Test set created")
    print(len(real_stock_price))
    
    # Getting the predicted stock price of 2017
    #print(type(training_set_1),type(real_stock_price))
    dataset_total = pd.concat((pd.Series(training_set_1), pd.Series(real_stock_price)), axis = 0)
    
    #print(len(training_set))
    #print(dataset_train['OPEN'])
    #print(len(real_stock_price))
    #print(dataset_test['OPEN'])
    inputs = dataset_total[len(dataset_total) - len(dataset_test) - timestamp:].values
    inputs = inputs.reshape(-1,1)
    inputs = sc.transform(inputs)
    
    X_test = []
    
    for i in range(timestamp, len(real_stock_price)+timestamp):
        X_test.append(inputs[i-timestamp:i, 0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    predicted_stock_price = regressor.predict(X_test)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)
    
    print("Stock values Predicted Successfully")
    #print(predicted_stock_price)

    #Finding Root mean Square error
    rms = sqrt(mean_squared_error(real_stock_price, predicted_stock_price))
    
    print("The Root Mean Square Error is : ",rms)
    
    # Visualising the results
    #print("Plotting Visualization")
    plt.plot(real_stock_price, color = 'red', label = 'Real ' + stock_name + ' Stock Price')
    plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted ' + stock_name +' Stock Price')
    plt.title(stock_name +' Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel(stock_name +' Stock Price')
    plt.legend()
    plt.show()

stock_list = ["549","20MICRONS","3IINFOTECH","3MINDIA","63MOONS","8KMILES","A2ZINFRA","AARTIDRUGS","AARTIIND","AARVEEDEN","ABAN","ABB","ABBOTINDIA","ABCIL","ABFRL","ABGSHIP","ABIRLANUVO","ABMINTLTD","ACC","ACCELYA","ACCORD","ACE","ACEINTEG","ACROPETAL","ADANIENT","ADANIPORTS","ADANIPOWER","ADANITRANS","ADFFOODS","ADHUNIK","ADHUNIKIND","ADI","ADLABS","ADORWELD","ADSL","ADVANIHOTR","ADVANTA","ADVENZYMES","AEGISCHEM","AFL","AGARIND","AGCNET","AGLSL","AGRITECH","AGRODUTCH","AGROPHOS","AHIMSA","AHLEAST","AHLUCONT","AHLWEST","AIAENG","AICHAMP","AIFL","AIL","AIRAN","AJANTPHARM","AJMERA","AKASH","AKSHARCHEM","AKSHOPTFBR","AKZOINDIA","ALANKIT","ALBERTDAVD","ALBK","ALCHEM","ALEMBICLTD","ALICON","ALKALI","ALKEM","ALKYLAMINE","ALLCARGO","ALLSEC","ALMONDZ","ALOKTEXT","ALPA","ALPHAGEO","ALPSINDUS","ALSTOMT&D","AMARAJABAT","AMBIKCO","AMBUJACEM","AMDIND","AMRUTANJAN","AMTEKAUTO","AMTL","ANANDAMRUB","ANANTRAJ","ANDHRABANK","ANDHRACEMT","ANDHRSUGAR","ANGIND","ANIKINDS","ANKITMETAL","ANSALAPI","ANSALHSG","ANTGRAPHIC","APARINDS","APCL","APCOTEXIND","APLAPOLLO","APLLTD","APOLLOHOSP","APOLLOTYRE","APOLSINHOT","APTECHT","ARCHIDPLY","ARCHIES","ARCOTECH","ARIES","ARIHANT","ARIHANTSUP","ARMANFIN","AROGRANITE","ARROWCOAT","ARROWGREEN","ARROWTEX","ARSHIYA","ARSSINFRA","ARTNIRMAN","ARVIND","ARVINDREM","ARVINFRA","ARVSMART","ASAHIINDIA","ASAHISONG","ASAL","ASHAPURMIN","ASHIANA","ASHIMASYN","ASHOKA","ASHOKLEY","ASIANHOTNR","ASIANPAINT","ASIANTILES","ASIL","ASLIND","ASPINWALL","ASSAMCO","ASTEC","ASTRAL","ASTRAMICRO","ASTRAZEN","ATFL","ATLANTA","ATLASCYCLE","ATNINTER","ATUL","ATULAUTO","AUBANK","AURDIS","AURIONPRO","AUROPHARMA","AUSOMENT","AUSTRAL","AUTOAXLES","AUTOIND","AUTOLITIND","AVADHSUGAR","AVANTIFEED","AVSL","AVTNPL","AXISBANK","AXISCADES","AXISGOLD","AXISNIFTY","AXISSCRGP","AYMSYNTEX","BAFNAPHARM","BAGFILMS","BAJAJ-AUTO","BAJAJCORP","BAJAJELEC","BAJAJFINSV","BAJAJHIND","BAJAJHLDNG","BAJFINANCE","BALAJITELE","BALAMINES","BALKRISHNA","BALKRISIND","BALLARPUR","BALMLAWRIE","BALPHARMA","BALRAMCHIN","BANARBEADS","BANARISUG","BANCOINDIA","BANG","BANKBARODA","BANKBEES","BANKINDIA","BANSAL","BANSWRAS","BARTRONICS","BASF","BASML","BATAINDIA","BAYERCROP","BBL","BBTC","BDR","BEARDSELL","BEDMUTHA","BEL","BEML","BEPL","BERGEPAINT","BFINVEST","BFUTILITIE","BGLOBAL","BGRENERGY","BHAGERIA","BHAGYANGR","BHAGYAPROP","BHAGYNAGAR","BHANDARI","BHARATFIN","BHARATFORG","BHARATGEAR","BHARATIDIL","BHARATRAS","BHARATWIRE","BHARTIARTL","BHARTISHIP","BHEL","BHUSANSTL","BIGBLOC","BIL","BILENERGY","BILPOWER","BINANIIND","BINDALAGRO","BIOCON","BIOFILCHEM","BIRLACABLE","BIRLACORPN","BIRLACOT","BIRLAERIC","BIRLAMONEY","BLBLIMITED","BLISSGVS","BLKASHYAP","BLS","BLUEBLENDS","BLUECHIP","BLUECOAST","BLUEDART","BLUESTARCO","BLUESTINFO","BNPCPGII","BODALCHEM","BOHRA","BOMDYEING","BOSCHLTD","BPCL","BPL","BRFL","BRIGADE","BRITANNIA","BROADCAST","BROOKS","BSE","BSELINFRA","BSL","BSLCAP21RG","BSLELFS5RG","BSLFEFS2RG","BSLFTPLVDG","BSLGOLDETF","BSLIMITED","BSLNIFTY","BSLRGES1RG","BSLRIFS1DG","BURNPUR","BUTTERFLY","BVCL","BYKE","C7DP","CADILAHC","CAIRN","CALSOFT","CAMLINFINE","CANBK","CANDC","CANFINHOME","CANTABIL","CAPF","CAPLIPOINT","CAPTRUST","CARBORUNIV","CAREERP","CARERATING","CASTEXTECH","CASTROLIND","CCCL","CCHHL","CCL","CDSL","CEATLTD","CEBBCO","CELEBRITY","CELESTIAL","CENTENKA","CENTEXT","CENTRALBK","CENTUM","CENTURYPLY","CENTURYTEX","CERA","CEREBRAINT","CESC","CGCL","CGPOWER","CHAMBLFERT","CHEMFALKAL","CHENNPETRO","CHOLAFIN","CHROMATIC","CIGNITITEC","CIMMCO","CINELINE","CINEVISTA","CIPLA","CKPPRODUCT","CLASSIC","CLEDUCATE","CLNINDIA","CLUTCHAUTO","CMAHENDRA","CMICABLES","CNOVAPETRO","COALINDIA","COCHINSHIP","COFFEEDAY","COLPAL","COMPINFO","COMPUSOFT","CONCOR","CONSOFINVT","CONTROLPR","CORALFINAC","CORDSCABLE","COREEDUTEC","COROMANDEL","CORPBANK","COSMOFILMS","COUNCODOS","COX&KINGS","CP7RG","CPSEETF","CREATIVE","CREATIVEYE","CREST","CRISIL","CRMFCP6RG","CRMFGETF","CROMPGREAV","CROMPTON","CROWN","CTE","CUB","CUBEXTUB","CUMMINSIND","CUPID","CURATECH","CYBERMEDIA","CYBERTECH","CYIENT","DAAWAT","DABUR","DALMIABHA","DALMIASUG","DAMODARIND","DATAMATICS","DBCORP","DBL","DBREALTY","DBSTOCKBRO","DCBBANK","DCM","DCMSHRIRAM","DCW","DECCANCE","DEEPAKFERT","DEEPAKNTR","DEEPIND","DELTACORP","DELTAMAGNT","DEN","DENABANK","DENORA","DEVIT","DFMFOODS","DHAMPURSUG","DHANBANK","DHANUKA","DHARSUGAR","DHFL","DHUNINV","DIAPOWER","DICIND","DIGJAM","DIGJAMLTD","DISHMAN","DISHTV","DIVISLAB","DLF","DLINKINDIA","DMART","DOLLAR","DOLPHINOFF","DONEAR","DPL","DPRMABGP","DPSCLTD","DQE","DREDGECORP","DRL","DRREDDY","DSKULKARNI","DSSL","DTIL","DUCON","DUNCANSLTD","DWARKESH","DWSFMP64RG","DWSMIDRD","DYNAMATECH","DYNATECH","DYNPRO","EASTSILK","EASUNREYRL","EBANK","ECEIND","ECLERX","ECLFINANCE","EDELWEISS","EDL","EDUCOMP","EHFLNCD","EICHERMOT","EIDPARRY","EIFFL","EIHAHOTELS","EIHOTEL","EIMCOELECO","EKC","ELAND","ELDERPHARM","ELECON","ELECTCAST","ELECTHERM","ELGIEQUIP","ELGIRUBCO","EMAMIINFRA","EMAMILTD","EMCO","EMKAY","EMKAYTOOLS","EMMBI","ENDURANCE","ENERGYDEV","ENGINERSIN","ENIL","ENTEGRA","EON","EQ30","EQUITAS","ERAINFRA","ERIS","EROSMEDIA","ESABINDIA","ESCORTS","ESL","ESSAROIL","ESSARSHPNG","ESSDEE","ESSELPACK","ESTER","EUROCERA","EUROMULTI","EUROTEXIND","EVEREADY","EVERESTIND","EVERONN","EXCEL","EXCELCROP","EXCELINDUS","EXIDEIND","FACT","FAGBEARING","FAIRCHEM","FARMAXIND","FCEL","FCL","FCONSUMER","FCSSOFT","FDC","FEDDERELEC","FEDDERLOYD","FEDERALBNK","FEL","FELDVR","FIEMIND","FILATEX","FINANTECH","FINCABLES","FINPIPE","FIRSTWIN","FLEXITUFF","FLFL","FMGOETZE","FMNL","FOCUS","FORTIS","FORTUNEFIN","FOSECOIND","FOURTHDIM","FRETAIL","FRL","FRLDVR","FSL","GABRIEL","GAEL","GAIL","GAL","GALLANTT","GALLISPAT","GAMMNINFRA","GAMMONIND","GANDHITUBE","GANECOS","GANESHHOUC","GANGESSECU","GANGOTRI","GARDENSILK","GARWALLROP","GATI","GAYAPROJ","GBNOV23","GDL","GEECEE","GEEKAYWIRE","GEINDSYS","GEMINI","GENESYS","GENUSPAPER","GENUSPOWER","GEOJITBNPP","GEOJITFSL","GEOMETRIC","GEPIL","GESHIP","GET&D","GHCL","GICHSGFIN","GICL","GILLANDERS","GILLETTE","GINNIFILA","GIPCL","GIRRESORTS","GISOLUTION","GITANJALI","GKWLIMITED","GLAXO","GLENMARK","GLFL","GLOBAL","GLOBALVECT","GLOBE","GLOBOFFS","GLOBUSSPR","GMBREW","GMDCLTD","GMRINFRA","GNA","GNFC","GOACARBON","GOCLCORP","GODFRYPHLP","GODREJCP","GODREJIND","GODREJPROP","GOENKA","GOKEX","GOKUL","GOKULAGRO","GOLDBEES","GOLDENTOBC","GOLDIAM","GOLDINFRA","GOLDIWIN","GOLDSHARE","GOLDTECH","GOODLUCK","GPIL","GPPL","GPTINFRA","GRANULES","GRAPHITE","GRASIM","GRAVITA","GREAVESCOT","GREENFIRE","GREENLAM","GREENPLY","GREENPOWER","GRETEX","GRINDWELL","GROBTEA","GRPLTD","GRUH","GSCLCEMENT","GSFC","GSKCONS","GSPL","GSS","GTL","GTLINFRA","GTNIND","GTNTEX","GTOFFSHORE","GTPL","GUFICBIO","GUJALKALI","GUJAPOLLO","GUJFLUORO","GUJGASLTD","GUJNRECOKE","GUJNREDVR","GULFOILLUB","GULFPETRO","GULPOLY","GVKPIL","H1190M36RG","H1190M36RQ","H1309S37RQ","HANUNG","HARITASEAT","HARRMALAYA","HATHWAY","HATSUN","HAVELLS","HBLPOWER","HBSTOCK","HCC","HCG","HCIL","HCL-INSYS","HCLTECH","HCPO36MORG","HCPO36MSRG","HDFC","HDFCBANK","HDFCMFGETF","HDFCNIFETF","HDFCSENETF","HDIL","HECPROJECT","HEG","HEIDELBERG","HELIOSMATH","HEOF1100RG","HERCULES","HERITGFOOD","HEROMOTOCO","HESTERBIO","HEXATRADEX","HEXAWARE","HFCL","HFEFRG","HGS","HIGHGROUND","HIKAL","HIL","HILTON","HIMATSEIDE","HINDALCO","HINDCOMPOS","HINDCOPPER","HINDDORROL","HINDMOTORS","HINDNATGLS","HINDOILEXP","HINDPETRO","HINDSYNTEX","HINDUJAFO","HINDUJAVEN","HINDUNILVR","HINDZINC","HIRECT","HISARMETAL","HITACHIHOM","HITECH","HITECHCORP","HITECHGEAR","HITECHPLAS","HMT","HMVL","HNGSNGBEES","HOCL","HONAUT","HONDAPOWER","HOTELEELA","HOTELRUGBY","HOVS","HPL","HRGESSRG1","HRGESSRG2","HSCL","HSIL","HTMEDIA","HUBTOWN","HUDCO","HUSYS","HUSYSLTD","IBREALEST","IBULHSGFIN","IBVENTURES","IBWSL","ICIBK1107","ICICI0202","ICICI0801","ICICI0901","ICICIBANK","ICICIPRULI","ICIL","ICNX100","ICRA","ICSA","IDBI","IDBIGOLD","IDEA","IDFC","IDFCBANK","IDFNIFTYET","IDR1AGD","IFBAGRO","IFBIND","IFCI","IFGLREFRAC","IGARASHI","IGL","IGOLD","IGPL","IIFCL","IIFL","IIFLFIN","IIHFL","IITL","IL&FSENGG","IL&FSTRANS","IMFA","IMIDCAP","IMPAL","IMPEXFERRO","INDBANK","INDHOTEL","INDIACEM","INDIAGLYCO","INDIANB","INDIANCARD","INDIANHUME","INDIGO","INDIGRID","INDLMETER","INDNIPPON","INDOCO","INDORAMA","INDOSOLAR","INDOTECH","INDOTHAI","INDOWIND","INDRAMEDCO","INDSWFTLAB","INDSWFTLTD","INDTERRAIN","INDUSINDBK","INEOSSTYRO","INFIBEAM","INFINITE","INFOBEAN","INFOMEDIA","INFRABEES","INFRATEL","INFY","INGERRAND","INIFTY","INNOIND","INOXLEISUR","INOXWIND","INSECTICID","INTEGRA","INTELLECT","INTENTECH","INV20","INVENTURE","IOB","IOC","IOLCP","IPAPPM","IPCALAB","IPRU2262","IPRU2288","IPRU2397","IPRU2599","IPRU2605","IPRU2660","IPRU2687","IPRU2691","IPRU2814","IRB","IRBINVIT","IREDA","IRFC","ISENSEX","ISFT","ISMTLTD","ITC","ITDC","ITDCEM","ITI","IVC","IVP","IVRCLINFRA","IVZINGOLD","IVZINNIFTY","IZMO","J&KBANK","JAGRAN","JAGSNPHARM","JAIBALAJI","JAICORPLTD","JAIHINDPRO","JAINSTUDIO","JALAN","JAMNAAUTO","JAYAGROGN","JAYBARMARU","JAYNECOIND","JAYSREETEA","JBCHEPHARM","JBFIND","JBMA","JCHAC","JCTEL","JENSONICOL","JETAIRWAYS","JETFREIGHT","JETKNIT","JHS","JIKIND","JINDALPHOT","JINDALPOLY","JINDALSAW","JINDALSTEL","JINDCOT","JINDRILL","JINDWORLD","JISLDVREQS","JISLJALEQS","JITFINFRA","JKCEMENT","JKIL","JKLAKSHMI","JKPAPER","JKTYRE","JMA","JMCPROJECT","JMFINANCIL","JMTAUTOLTD","JNPT","JOCIL","JPASSOCIAT","JPINFRATEC","JPOLYINVST","JPPOWER","JSL","JSLHISAR","JSWENERGY","JSWHL","JSWSTEEL","JUBILANT","JUBLFOOD","JUBLINDS","JUNIORBEES","JUSTDIAL","JVLAGRO","JYOTHYLAB","JYOTISTRUC","KABRAEXTRU","KAJARIACER","KAKATCEM","KALINDEE","KALPATPOWR","KALYANIFRG","KAMATHOTEL","KAMDHENU","KANANIIND","KANORICHEM","KANSAINER","KARMAENG","KARURVYSYA","KAUSHALYA","KAVVERITEL","KAWIRES","KAYA","KCP","KCPSUGIND","KDDL","KEC","KECL","KEERTI","KEI","KELLTONTEC","KEMROCK","KERNEX","KESARENT","KESORAMIND","KEYCORPSER","KGL","KHAITANELE","KHAITANLTD","KHANDSE","KICL","KIL","KILITCH","KINGFA","KIOCL","KIRIINDUS","KIRLOSBROS","KIRLOSENG","KIRLOSIND","KITEX","KKCL","KKVAPOW","KMSUGAR","KNRCON","KOHINOOR","KOKUYOCMLN","KOLTEPATIL","KOPRAN","KOTAKBANK","KOTAKBKETF","KOTAKGOLD","KOTAKNIFTY","KOTAKNV20","KOTAKPSUBK","KOTARISUG","KOTHARIPET","KOTHARIPRO","KPIT","KPRMILL","KRBL","KREBSBIO","KRIDHANINF","KRISHANA","KSBPUMPS","KSCL","KSERASERA","KSK","KSL","KTIL","KTKBANK","KWALITY","L&TFH","L&TFINANCE","L&TINFRA","LAKPRE","LAKSHMIEFL","LAKSHVILAS","LALPATHLAB","LAMBODHARA","LAOPALA","LAURUSLABS","LAXMICOT","LAXMIMACH","LCCINFOTEC","LEEL","LEXUS","LFIC","LGBBROSLTD","LGBFORGE","LIBAS","LIBERTSHOE","LICHSGFIN","LICNETFGSC","LICNETFN50","LICNETFSEN","LICNFNHGP","LINCOLN","LINCPEN","LINDEINDIA","LIQUIDBEES","LITL","LLOYDELENG","LML","LOKESHMACH","LOTUSEYE","LOVABLE","LOWVOLIWIN","LPDC","LSIL","LT","LTEBFG","LTI","LTTS","LUMAXAUTO","LUMAXIND","LUMAXTECH","LUPIN","LUXIND","LYCOS","LYKALABS","LYPSAGEMS","M&M","M&MFIN","M100","M14RG","M15RG","M50","MAANALU","MADHAV","MADHUCON","MADRASFERT","MAGADSUGAR","MAGMA","MAGNUM","MAHABANK","MAHASTEEL","MAHESHWARI","MAHINDCIE","MAHLIFE","MAHSCOOTER","MAHSEAMLES","MAITHANALL","MAJESCO","MALUPAPER","MALWACOTT","MANAKALUCO","MANAKCOAT","MANAKINDST","MANAKSIA","MANAKSTEEL","MANALIPETC","MANAPPURAM","MANDHANA","MANGALAM","MANGCHEFER","MANGLMCEM","MANGTIMBER","MANINDS","MANINFRA","MANPASAND","MANUGRAPH","MARALOVER","MARATHON","MARICO","MARKSANS","MARUTI","MASKINVEST","MASTEK","MAWANASUG","MAX","MAXINDIA","MAXVIL","MAXWELL","MAYURUNIQ","MAZDA","MBAPL","MBECL","MBLINFRA","MC6RD","MCDHOLDING","MCDOWELL-N","MCL","MCLEODRUSS","MCX","MEGASOFT","MEGH","MELSTAR","MENONBE","MEP","MERCATOR","MERCK","METALFORGE","METKORE","MFSL","MGL","MHRIL","MIC","MICROSEC","MIDCAPIWIN","MINDACORP","MINDAIND","MINDTECK","MINDTREE","MIRCELECTR","MIRZAINT","MITCON","MKPL","MMFL","MMNL","MMTC","MOHINI","MOHITIND","MOHOTAMILL","MOIL","MOLDTECH","MOLDTKPAC","MOMAI","MONNETISPA","MONSANTO","MONTECARLO","MORARJEE","MOREPENLAB","MOSERBAER","MOTHERSUMI","MOTILALOFS","MOTOGENFIN","MPHASIS","MPSLTD","MRF","MRO-TEK","MRPL","MSPL","MTEDUCARE","MTNL","MUKANDENGG","MUKANDLTD","MUKTAARTS","MUNJALAU","MUNJALSHOW","MURLIIND","MURUDCERA","MUTHOOTCAP","MUTHOOTFIN","MVL","MYSOREBANK","N100","NABARD","NAGAAGRI","NAGAFERT","NAGAROIL","NAGREEKCAP","NAGREEKEXP","NAHARCAP","NAHARINDUS","NAHARPOLY","NAHARSPING","NAKODA","NANDANI","NATCOPHARM","NATHBIOGEN","NATIONALUM","NATNLSTEEL","NAUKRI","NAVINFLUOR","NAVKARCORP","NAVNETEDUL","NBCC","NBIFIN","NBVENTURES","NCC","NCLIND","NCOPPER","NDGL","NDL","NDTV","NECCLTD","NECLIFE","NELCAST","NELCO","NEPCMICON","NESCO","NESTLEIND","NET4","NETWORK18","NEULANDLAB","NEXTMEDIA","NEYVELILIG","NFL","NGCT","NH","NHAI","NHBTF2014","NHBTF2023","NHPC","NIBL","NICCO","NIF100IWIN","NIFTYBEES","NIFTYEES","NIFTYIWIN","NIITLTD","NIITTECH","NILAINFRA","NILKAMAL","NIPPOBATRY","NIRVIKARA","NITCO","NITESHEST","NITINFIRE","NITINSPIN","NITIRAJ","NKIND","NLCINDIA","NMDC","NOCIL","NOESISIND","NOIDATOLL","NORBTEAEXP","NRAIL","NRBBEARING","NSIL","NTL","NTPC","NUCLEUS","NUTEK","NV20IWIN","OBEROIRLTY","OCCL","OCL","OFSS","OIL","OILCOUNTUB","OISL","OMAXAUTO","OMAXE","OMKARCHEM","OMMETALS","OMNITECH","ONELIFECAP","ONGC","ONMOBILE","ONWARDTEC","OPAL","OPTIEMUS","OPTOCIRCUI","ORBITCORP","ORBTEXP","ORCHIDPHAR","ORICONENT","ORIENTABRA","ORIENTALTL","ORIENTBANK","ORIENTBELL","ORIENTCEM","ORIENTHOT","ORIENTLTD","ORIENTPPR","ORIENTREF","ORISSAMINE","ORTEL","ORTINLABSS","OUDHSUG","PAEL","PAGEIND","PALASHSECU","PALREDTEC","PALREDTECH","PANACEABIO","PANACHE","PANAMAPET","PANORAMUNI","PANSARI","PAPERPROD","PARABDRUGS","PARACABLES","PARAGMILK","PARAPRINT","PARASPETRO","PARRYSUGAR","PARSVNATH","PATELENG","PATINTLOG","PATSPINLTD","PBAINFRA","PCJEWELLER","PDMJEPAPER","PDPL","PDSMFL","PDUMJEIND","PDUMJEPULP","PEARLPOLY","PEL","PENIND","PENINLAND","PENPEBS","PERFECT","PERSISTENT","PETRONENGG","PETRONET","PFC","PFIZER","PFOCUS","PFRL","PFS","PGEL","PGHH","PGIL","PHILIPCARB","PHOENIXLL","PHOENIXLTD","PIDILITIND","PIIND","PILANIINVS","PILITA","PINCON","PIONDIST","PIONEEREMB","PIPAVAVDOC","PIRPHYTO","PITTILAM","PKTEA","PLASTIBLEN","PLETHICO","PNB","PNBGILTS","PNBHOUSING","PNC","PNCINFRA","PNEUMATIC","POCHIRAJU","PODDARHOUS","PODDARMENT","POKARNA","POLARIS","POLYMED","POLYPLEX","PONNIERODE","POWERGRID","POWERMECH","PPAP","PRABHAT","PRADIP","PRAENG","PRAJIND","PRAKASH","PRAKASHCON","PRAKASHSTL","PRATIBHA","PRECAM","PRECOT","PRECWIRE","PREMEXPLN","PREMIER","PREMIERPOL","PRESSMN","PRESTIGE","PRICOL","PRICOLLTD","PRIMESECU","PRISMCEM","PROLIFE","PROVOGE","PROZONINTU","PSB","PSL","PSPPROJECT","PSUBNKBEES","PTC","PTL","PUNJABCHEM","PUNJLLOYD","PURVA","PUSHPREALM","PVP","PVR","QGOLDHALF","QNIFTY","QUESS","QUICKHEAL","QUINTEGRA","RADAAN","RADICO","RADIOCITY","RAIN","RAINBOWPAP","RAIREKMOH","RAJESHEXPO","RAJOIL","RAJRAYON","RAJSREESUG","RAJTV","RAJVIR","RALLIS","RAMANEWS","RAMASTEEL","RAMCOCEM","RAMCOIND","RAMCOSYS","RAMKY","RAMSARUP","RANASUG","RANEENGINE","RANEHOLDIN","RASOYPR","RATNAMANI","RAYMOND","RBL","RBLBANK","RCF","RCOM","RDAFTIIIDG","RDEL","RECLTD","REDINGTON","REFEX","REGENCERAM","REIAGROLTD","REISIXTEN","RELAXO","RELBANK","RELCAPITAL","RELCNX100","RELCONS","RELDIVOPP","RELGOLD","RELGRNIFTY","RELIANCE","RELIFIN","RELIGARE","RELIGAREGO","RELINFRA","RELNIFTY","RELNV20","REMSONSIND","RENUKA","REPCOHOME","REPRO","RESPONIND","REVATHI","RHFL","RICOAUTO","RIIL","RJL","RKDL","RKFORGE","RMCL","RML","RMMIL","ROHITFERRO","ROHLTD","ROLLT","ROLTA","ROSSELLIND","RPGLIFE","RPOWER","RPPINFRA","RRSLGETF","RSSOFTWARE","RSWM","RSYSTEMS","RTNINFRA","RTNPOWER","RUBYMILLS","RUCHINFRA","RUCHIRA","RUCHISOYA","RUPA","RUSHIL","SABEVENTS","SABTN","SADBHAV","SADBHIN","SAGARDEEP","SAGCEM","SAIL","SAKAR","SAKHTISUG","SAKSOFT","SAKUMA","SALASAR","SALONA","SALORAINTL","SALSTEEL","SALZERELEC","SAMBHAAV","SAMTEL","SANCO","SANDESH","SANGAMIND","SANGHIIND","SANGHVIFOR","SANGHVIMOV","SANGINITA","SANOFI","SANWARIA","SARDAEN","SAREGAMA","SARLAPOLY","SASKEN","SASTASUNDR","SATHAISPAT","SATIN","SBBJ","SBIN","SBT","SC2RD","SC3RG","SCHAEFFLER","SCHAND","SCHNEIDER","SCI","SDAFIIIGR","SDBL","SEAMECLTD","SEINV","SELAN","SELMCL","SENSEXIWIN","SEOFIVGR","SEPOWER","SEQUENT","SERVALL","SERVOTECH","SESHAPAPER","SETCO","SETF10GILT","SETFGOLD","SETFNIF50","SETFNIFBK","SETFNN50","SEZAL","SFCL","SFL","SGBAUG24","SGBFEB24","SGBJUL25","SGBMAR24","SGBMAR25","SGBMAY25","SGBNOV23","SGBNOV24","SGBSEP24","SGFL","SGJHL","SGL","SHAHALLOYS","SHAIVAL","SHAKTIPUMP","SHALPAINTS","SHANKARA","SHANTI","SHANTIGEAR","SHARDACROP","SHARDAMOTR","SHARIABEES","SHARONBIO","SHEMAROO","SHILPAMED","SHILPI","SHIRPUR-G","SHIVAMAUTO","SHIVTEX","SHK","SHLAKSHMI","SHOPERSTOP","SHREECEM","SHREEPUSHK","SHREERAMA","SHRENIK","SHRENUJ","SHREYANIND","SHREYAS","SHRIASTER","SHRIPISTON","SHRIRAMCIT","SHRIRAMEPC","SHYAMCENT","SHYAMTEL","SICAGEN","SICAL","SIEMENS","SIGNET","SIIL","SIKKO","SIL","SILINV","SIMBHALS","SIMPLEX","SIMPLEXINF","SINTEX","SIRPAPER","SIS","SITASHREE","SITICABLE","SITINET","SIYSIL","SJVN","SKFINDIA","SKIL","SKIPPER","SKMEGGPROD","SKSMICRO","SMARTLINK","SMCSRIIIRD","SMCSRVIRD","SMCSRVIRG","SMCSRVRD","SMCSRVRG","SMLISUZU","SMPL","SMSLIFE","SMSPHARMA","SNOWMAN","SOBHA","SOFTTECHGR","SOLARINDS","SOMANYCERA","SOMATEX","SOMICONVEY","SONASTEER","SONATSOFTW","SORILHOLD","SORILINFRA","SOTL","SOUTHBANK","SPAL","SPARC","SPCENET","SPECIALITY","SPECTACLE","SPENTEX","SPHEREGSL","SPIC","SPICEMOBI","SPLIL","SPMLINFRA","SPTL","SPYL","SQSBFSI","SREEL","SREIBNPNCD","SREINFRA","SRF","SRGINFOTEC","SRHHYPOLTD","SRIPIPES","SRIRAM","SRSLTD","SRTRANSFIN","SSWL","STAMPEDE","STAN","STAR","STARCEMENT","STARPAPER","STCINDIA","STEELCITY","STEELXIND","STEL","STERLINBIO","STERTOOLS","STINDIA","STOREONE","STRTECH","STYABS","SUBEX","SUBROS","SUDAR","SUDARSCHEM","SUJANATWR","SUJANAUNI","SUMEETINDS","SUMMITSEC","SUNCLAYLTD","SUNDARAM","SUNDARMFIN","SUNDRMBRAK","SUNDRMFAST","SUNFLAG","SUNILHITEC","SUNPHARMA","SUNTECK","SUNTV","SUPERHOUSE","SUPERSPIN","SUPPETRO","SUPRAJIT","SUPREMEIND","SUPREMEINF","SUPREMETEX","SURANACORP","SURANAIND","SURANASOL","SURANAT&P","SUREVIN","SURYAJYOTI","SURYALAXMI","SURYAROSNI","SUTLEJTEX","SUVEN","SUZLON","SVOGL","SWANENERGY","SWARAJENG","SWELECTES","SYMPHONY","SYNCOM","SYNDIBANK","SYNGENE","TAINWALCHM","TAJGVK","TAKE","TALBROAUTO","TALWALKARS","TANLA","TANTIACONS","TARAJEWELS","TARAPUR","TARMAT","TASTYBITE","TATACHEM","TATACOFFEE","TATACOMM","TATAELXSI","TATAGLOBAL","TATAINVEST","TATAMETALI","TATAMOTORS","TATAMTRDVR","TATAPOWER","TATASPONGE","TATASTEEL","TBZ","TCFSL","TCI","TCIDEVELOP","TCIEXP","TCIFINANCE","TCS","TDPOWERSYS","TEAMLEASE","TECHIN","TECHM","TECHNO","TECHNOFAB","TECPRO","TEJASNET","TERASOFT","TEXINFRA","TEXMOPIPES","TEXRAIL","TFCILTD","TFL","TGBHOTELS","THANGAMAYL","THEJO","THEMISMED","THERMAX","THIRUSUGAR","THOMASCOOK","THOMASCOTT","THYROCARE","TI","TIDEWATER","TIIL","TIJARIA","TIL","TIMESGTY","TIMETECHNO","TIMKEN","TINPLATE","TIPSINDLTD","TIRUMALCHM","TITAN","TMRVL","TNPETRO","TNPL","TNTELE","TODAYS","TOKYOPLAST","TORNTPHARM","TORNTPOWER","TOTAL","TPLPLASTEH","TRANSWIND","TREEHOUSE","TRENT","TRF","TRICOM","TRIDENT","TRIGYN","TRIL","TRITURBINE","TRIVENI","TTKHLTCARE","TTKPRESTIG","TTL","TTML","TUBEINVEST","TULSI","TV18BRDCST","TVSELECT","TVSMOTOR","TVSSRICHAK","TVTODAY","TVVISION","TWL","UBHOLDINGS","UBL","UCALFUEL","UCOBANK","UFLEX","UFO","UGARSUGAR","UJAAS","UJJIVAN","ULTRACEMCO","UMANGDAIRY","UMESLTD","UNICHEMLAB","UNIENTER","UNIONBANK","UNIPLY","UNITECH","UNITEDBNK","UNITEDPOLY","UNITEDTEA","UNITY","UNIVASTU","UNIVCABLES","UPERGANGES","UPL","URJA","USHAMART","USHERAGRO","UTCPOSRGR1","UTIFEFRDP1","UTIFEFRDP2","UTIFEFRGR1","UTIFEFRGR2","UTINEXT50","UTINIFTETF","UTISENSETF","UTRGESSRGR","UTRGRXIX20","UTTAMSTL","UTTAMSUGAR","UTTAMVALUE","UVSL","V2RETAIL","VADILALIND","VAIBHAVGBL","VAISHALI","VAKRANGEE","VALECHAENG","VALUEIND","VARDHACRLC","VARDMNPOLY","VASCONEQ","VASWANI","VBL","VEDL","VENKEYS","VENUSREM","VESUVIUS","VETO","VF1RD","VF1RG","VGUARD","VHL","VICEROY","VIDEOIND","VIDHIDYE","VIDHIING","VIJAYABANK","VIJIFIN","VIJSHAN","VIKASECO","VIMALOIL","VIMTALABS","VINATIORGA","VINDHYATEL","VINYLINDIA","VIPCLOTHNG","VIPIND","VIPULLTD","VISAKAIND","VISASTEEL","VISESHINFO","VISHNU","VISUINTL","VIVIDHA","VIVIMEDLAB","VKSPL","VLSFINANCE","VMART","VOLTAMP","VOLTAS","VRLLOG","VSCL","VSSL","VSTIND","VSTTILLERS","VTL","WABAG","WABCOINDIA","WALCHANNAG","WANBURY","WB3RD","WB3RG","WEALTH","WEBELSOLAR","WEIZFOREX","WEIZMANIND","WELCORP","WELENT","WELINV","WELSPUNIND","WELSYNTEX","WENDT","WHEELS","WHIRLPOOL","WILLAMAGOR","WINDMACHIN","WINSOME","WIPL","WIPRO","WOCKPHARMA","WONDERLA","WSI","WSTCSTPAPR","XCHANGING","XLENERGY","XPROINDIA","YESBANK","ZANDUREALT","ZEEL","ZEELEARN","ZEEMEDIA","ZENITHBIR","ZENITHCOMP","ZENITHEXPO","ZENSARTECH","ZENTEC","ZICOM","ZODIACLOTH","ZODJRDMKJ","ZOTA","ZUARI","ZUARIGLOB","ZYDUSWELL","ZYLOG"]


def predict():
    while(True):
        stock_name = input("Please enter the Stock Name you want to predict OR q to Exit: ")
        counter  = 0
        if(stock_name == 'q' or stock_name == "Q"):
            break
        for i in stock_list:
            if(i == stock_name.upper()):
                predict_algorithm(stock_name.upper())
                counter = 0
                break
            else:
                counter += 1
        if(counter > 1):
            print("\nYou have entered an incorrect Stock Name. Please enter correct Stock Name!!")
            counter = 0
    
predict()
